In [15]:
import cv2
import numpy as np

input_image_path = "test/testvazo.jpg"  
output_image_path = "ciktilar/iyilestirilmis.jpg"  

#görsellerde pixel arttırma ve kenar keskinleştirme uyguluyor(eğer görsel yüksekliği 720 pikseldden fazlaysa piksel arttırma adımını atlar)
def enhance_image(input_image_path, output_image_path, scale_factor=2):
    try:
        
        img = cv2.imread(input_image_path)

        if img.shape[0] > 720:
            print("Görsel yüksekliği 720 pikseldan fazla. Yeniden ölçeklendirme yapılmayacak.")
            enhanced_img = img
        else:
            # Görseli yeniden ölçeklendirme
            new_size = (int(img.shape[1] * scale_factor), int(img.shape[0] * scale_factor))
            enhanced_img = cv2.resize(img, new_size, interpolation=cv2.INTER_CUBIC)

        # Görseli netleştirme (Kenar belirginleştirme)
        kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])
        enhanced_img = cv2.filter2D(enhanced_img, -1, kernel)

        cv2.imwrite(output_image_path, enhanced_img)
        print(f"İyileştirilmiş görsel kaydedildi: {output_image_path}")

    except Exception as e:
        print(f"Görsel iyileştirme sırasında bir hata oluştu: {e}")

enhance_image(input_image_path, output_image_path)


Görsel yüksekliği 720 pikseldan fazla. Yeniden ölçeklendirme yapılmayacak.
İyileştirilmiş görsel kaydedildi: ciktilar/iyilestirilmis.jpg


In [16]:
import subprocess

#görüntünün arka planını backgroundremover aracı kullanarak kaldırıyor
input_image_path = "ciktilar/iyilestirilmis.jpg"
output_image_path = "ciktilar/arkaplansiz.jpg"


command = ["backgroundremover", "-i", input_image_path, "-o", output_image_path]


subprocess.run(command)

ciktilar/arkaplansiz.jpg
DEBUG: path to be checked: /home/jupyter/.u2net/u2net.pth


CompletedProcess(args=['backgroundremover', '-i', 'ciktilar/iyilestirilmis.jpg', '-o', 'ciktilar/arkaplansiz.jpg'], returncode=0)

In [17]:
from PIL import Image
import os

#arkaplanı silinmiş olan görsele seçilen rgb renk arkapan olarak eklenir 
foreground_path = "ciktilar/arkaplansiz.jpg"
output_path = "ciktilar/beyazarkaplan"

rgb_input = "256,256,256"#beyaz
background_color = tuple(map(int, rgb_input.split(",")))


foreground = Image.open(foreground_path)


if foreground.mode != 'RGBA':
    foreground = foreground.convert('RGBA')


background = Image.new("RGBA", (1024, 1024), background_color + (255,))#arkaplan boyutu 1024x1024


foreground = foreground.resize((1024, 1024), Image.ANTIALIAS)

combined_image = Image.alpha_composite(background, foreground)

file_extension = os.path.splitext(foreground_path)[-1].lower()

if file_extension == ".jpg" or file_extension == ".jpeg":

    combined_image = combined_image.convert("RGB")
    output_path += ".jpg"
else:
    output_path += ".png"

combined_image.save(output_path)

print("Yeni arka planla görsel oluşturuldu:", output_path)



Yeni arka planla görsel oluşturuldu: ciktilar/beyazarkaplan.jpg


/var/tmp/ipykernel_252723/1757584265.py:22: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  foreground = foreground.resize((1024, 1024), Image.ANTIALIAS)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#görsel önccelikle gri tonlarına çevrilir. Obje dikddörgen çerçeve içerisine alınır ve açısı hesaplanır ve obje dik konuma getirilir(eğer açı 7 dereceden küçükse görüntü aynen kalır). 
image_path = 'ciktilar/beyazarkaplan.jpg'  
image = cv2.imread(image_path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


if len(contours) == 0:
    angle = 0
else:

    largest_contour = max(contours, key=cv2.contourArea)

    if len(largest_contour) >= 5:

        ellipse = cv2.fitEllipse(largest_contour)
        angle = ellipse[-1]  # Elipsin açısını al
    else:
        angle = 0


if abs(angle) < 7:#sınır açısı
    output_image = image.copy()  
else:

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Boş alanları beyaz ile doldurarak görüntüyü döndür
    output_image = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))


output_path = 'ciktilar/dogruaci.jpg'  # Kaydedilecek dosya adı ve yolu
cv2.imwrite(output_path, output_image)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Orijinal Görüntü")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

plt.subplot(1, 2, 2)
plt.title("Düzeltilmiş Görüntü (fitEllipse ile)")
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))

plt.tight_layout()
plt.show()


In [19]:
import subprocess

#görüntünün arka planını backgroundremover aracı kullanarak kaldırıyor
input_image_path = "ciktilar/dogruaci.jpg"
output_image_path = "ciktilar/gorsel_yeni_arkaplansiz.jpg"

command = ["backgroundremover", "-i", input_image_path, "-o", output_image_path]

subprocess.run(command)

ciktilar/gorsel_yeni_arkaplansiz.jpg
DEBUG: path to be checked: /home/jupyter/.u2net/u2net.pth


CompletedProcess(args=['backgroundremover', '-i', 'ciktilar/dogruaci.jpg', '-o', 'ciktilar/gorsel_yeni_arkaplansiz.jpg'], returncode=0)

In [13]:
rgb="150,150,150"

In [14]:
from PIL import Image
import os

#arkaplanı silinmiş olan görsele seçilen rgb renk arkapan olarak eklenir 
foreground_path = "ciktilar/gorsel_yeni_arkaplansiz.jpg"
output_path = "ciktilar/gorsel_yeni_arkaplan"

rgb_input = rgb
background_color = tuple(map(int, rgb_input.split(",")))


foreground = Image.open(foreground_path)


if foreground.mode != 'RGBA':
    foreground = foreground.convert('RGBA')


background = Image.new("RGBA", (1024, 1024), background_color + (255,))

foreground = foreground.resize((1024, 1024), Image.ANTIALIAS)


combined_image = Image.alpha_composite(background, foreground)


file_extension = os.path.splitext(foreground_path)[-1].lower()

if file_extension == ".jpg" or file_extension == ".jpeg":

    combined_image = combined_image.convert("RGB")
    output_path += ".jpg"
else:
    output_path += ".png"


combined_image.save(output_path)

print("Yeni arka planla görsel oluşturuldu:", output_path)

Yeni arka planla görsel oluşturuldu: ciktilar/gorsel_yeni_arkaplan.jpg


/var/tmp/ipykernel_252723/1268166566.py:22: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  foreground = foreground.resize((1024, 1024), Image.ANTIALIAS)


In [16]:
!pip install flask

     |████████████████████████████████| 101 kB 1.6 MB/s ta 0:00:01
